In [10]:
import platform

platform.python_version()

'3.11.3'

## 1. 프로세스 알아보기 

## 1-1. 멀티 프로세스 모듈 

In [26]:
import multiprocessing
import time
import os

## 모듈 내의 속성 확인하기 

In [6]:
for i in dir(multiprocessing) :
    if not i.startswith("_") :
        print(i, end=", ")

Array, AuthenticationError, Barrier, BoundedSemaphore, BufferTooShort, Condition, Event, JoinableQueue, Lock, Manager, Pipe, Pool, Process, ProcessError, Queue, RLock, RawArray, RawValue, SUBDEBUG, SUBWARNING, Semaphore, SimpleQueue, TimeoutError, Value, active_children, allow_connection_pickling, context, cpu_count, current_process, freeze_support, get_all_start_methods, get_context, get_logger, get_start_method, log_to_stderr, parent_process, process, reducer, reduction, set_executable, set_forkserver_preload, set_start_method, sys, 

## 현재 cpu 갯수 

In [2]:
multiprocessing.cpu_count()

10

### 현재 프로세스 확인하기 

In [17]:
multiprocessing.current_process()

<_MainProcess name='MainProcess' parent=None started>

In [8]:
multiprocessing.parent_process()

## 1-2 부모와 자식 프로세스 처리 

- 멀티프로세스 환경에서는 부모 프로세스와 자식 프로세스가 각각 독립적인 공간에서 실행됩니다. 
- 부모와 자식 간의 통신 및 상호작용을 위해 파이썬은 multiprocessing 모듈을 제공합니다. 

### 프로세스간 충돌을 방지하기 위해 함수를 별도 모듈로 처리 

In [25]:
%%writefile child_process.py
import multiprocessing
import time
import os

def child_process():
    print("Child process is running")
    print(" child process " ,multiprocessing.current_process())
    print(" parant process ", multiprocessing.parent_process())
    print(f"Child process (PID: {os.getpid()}) is running")
    time.sleep(2)
    print("Child process completed")

Overwriting child_process.py


### 메인프로세스에 자식프로세스 처리

In [27]:
from child_process import child_process


if __name__ == "__main__":
    
    print(" parent process " ,multiprocessing.current_process())
    print(f"parent process (PID: {os.getpid()}) is running")
    # 부모 프로세스에서 자식 프로세스 생성
    child = multiprocessing.Process(target=child_process)

    # 자식 프로세스 시작
    child.start()

    print("Parent process is running")

    # 자식 프로세스가 완료될 때까지 대기
    child.join()

    print("Parent process completed")


 parent process  <_MainProcess name='MainProcess' parent=None started>
parent process (PID: 82511) is running
Parent process is running
Child process is running
 child process  <Process name='Process-34' parent=82511 started>
 parant process  <_ParentProcess name='MainProcess' parent=None unknown>
Child process (PID: 93052) is running
Child process completed
Parent process completed


## 1-3 프로세스 시작과 종료 처리 

- Process 클래스를 사용하여 새로운 프로세스를 생성할 수 있습니다. 
- 생성한 프로세스는 독립적으로 실행되며, 별도의 메모리 공간을 가지고 있습니다.

### 프로세스에서 작업할  함수를 별도 모듈로 작성 

In [20]:
%%writefile worker00.py

def my_function():
    print("Hello from another process!")


Overwriting worker00.py


### 프로세스를 생성


In [21]:
from multiprocessing import Process
from worker00 import my_function

In [22]:
# 새로운 프로세스 생성
my_process = Process(target=my_function)


### 프로세스 시작 

- start() 메서드를 사용하여 프로세스를 시작하고, 내부적으로 target으로 지정한 함수 또는 메서드를 실행합니다.

In [23]:
my_process.start()  # 프로세스 시작
my_process.join()   # 프로세스가 완료될 때까지 대기

Hello from another process!


## 2. 멀티프로세스 활용  

## 2-1 프로세스 간 통신: 

- 멀티프로세스 환경에서는 프로세스 간에 데이터를 공유하거나 통신할 수 있어야 합니다. 
- multiprocessing 모듈은 이러한 통신을 지원하는 여러 메커니즘을 제공합니다.

In [1]:
%%writefile worker01.py

def square_numbers(numbers, result, index):
    for idx, num in enumerate(numbers):
        result[idx] = num * num
        index.value = idx  # 공유 메모리 업데이트
        

def square_number(number):
    return number * number

Overwriting worker01.py


In [37]:
from multiprocessing import Process, Value

from worker01 import square_numbers

numbers = [1, 2, 3, 4, 5]
result = Array('i', [0] * len(numbers))  # 정수형 배열 공유 메모리
index = Array('i', [-1])                 # 정수형 배열 공유 메모리

my_process = Process(target=square_numbers, args=(numbers, result, index))
my_process.start()
my_process.join()

print(list(result))
print(index[0])


[1, 4, 9, 16, 25]
-1


## 2-2 멀티프로세스 처리를 위해서는 두 개의 모듈로 구분 

- 게산하는 함수를 별도로 정의 

In [14]:
from multiprocessing import Process, Array

### 실행할 함수를 별도의 모듈에 생성

In [27]:
%%writefile worker.py
# worker.py


def square_numbers(numbers, result, index):
    for idx, num in enumerate(numbers):
        result[idx] = num * num
        index.value = idx                   # 공유 메모리 업데이트

Overwriting worker.py


### 메인 프로세스 처리 

- multiprocessing.Array를 사용하여 배열을 만들고, 해당 배열을 공유 메모리로 사용하는 방법이 있습니다.

In [28]:
from multiprocessing import Process, Array
from worker import square_numbers

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5]
    result = Array('i', [0] * len(numbers))  # 정수형 배열 공유 메모리
    index = Array('i', [-1])                 # 정수형 배열 공유 메모리

    # 멀티프로세스 생성
    process = Process(target=square_numbers, args=(numbers, result, index))

    # 프로세스 시작
    process.start()

    # 프로세스가 완료될 때까지 대기
    process.join()

    # 결과 출력
    print(list(result))
    print(index[0])



[1, 4, 9, 16, 25]
-1


## 2-3  2 개의 프로세스에서 처리 

In [6]:
%%writefile task.py
import time

def task1():
    for _ in range(5):
        print("Task 1 is running")
        time.sleep(1)

def task2():
    for _ in range(5):
        print("Task 2 is running")
        time.sleep(1)

Overwriting task.py


In [7]:
from multiprocessing import Process
from task import task1, task2

if __name__ == "__main__":
    # 두 개의 프로세스 생성
    process1 = Process(target=task1)
    process2 = Process(target=task2)

    # 각각의 프로세스 시작
    process1.start()
    process2.start()

    # 각각의 프로세스가 완료될 때까지 대기
    process1.join()
    process2.join()

    print("Both tasks are completed")


Task 1 is running
Task 1 is running
Task 1 is running
Task 1 is running
Task 1 is running
Task 2 is running
Task 2 is running
Task 2 is running
Task 2 is running
Task 2 is running
Both tasks are completed


## 2-4 프로세스 풀에서 처리 

In [8]:
from multiprocessing import Pool

from worker01 import square_number

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5]

    # Pool 생성 (기본적으로 CPU 코어 수 만큼의 프로세스를 사용)
    pool = Pool()

    # map 함수를 사용하여 각 숫자를 제곱하는 작업을 병렬로 처리
    result = pool.map(square_number, numbers)

    # 프로세스 풀 종료 및 대기
    pool.close()
    pool.join()

    # 결과 출력
    print(result)

[1, 4, 9, 16, 25]


In [9]:
from multiprocessing import Pool

from worker01 import square_number

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5]

    # Pool 생성 (기본적으로 CPU 코어 수 만큼의 프로세스를 사용)
    with Pool() as pool:
        # map 함수를 사용하여 각 숫자를 제곱하는 작업을 병렬로 처리
        result = pool.map(square_number, numbers)

    # 결과 출력
    print(result)

[1, 4, 9, 16, 25]
